In [1]:
import random
import sys
from typing import Optional, List
from configparser import ConfigParser
import time
import numpy as np

import carla
import agents
from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO
from agents.navigation.global_route_planner import GlobalRoutePlanner

import carla
def draw_waypoints(
    world: carla.libcarla.World,
    waypoints: List[carla.libcarla.Waypoint],
    life_time: float = 10.0,  # Seconds
    color: List[int] = [0, 255, 0],
) -> None:
    """
    Draws a list of waypoints in the given CARLA world.
    """
    if waypoints in [[], None]: return
    gamma = 1
    for waypoint in waypoints:
        color = list((np.array(color)*gamma).astype(np.uint16))
        gamma = gamma * 0.95
        world.debug.draw_string(
            waypoint.transform.location,
            "O",
            draw_shadow=False,
            color=carla.Color(r=int(color[0]), g=int(color[1]), b=int(color[2])),
            life_time=life_time,
            persistent_lines=True,
        )

def get_client() -> carla.libcarla.Client:
    """
    Get a CARLA client object. The client object enables interaction with the CARLA server.
    """
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)
    print("Connection to CARLA server established!")
    return client

def get_global_planner(
    world: carla.libcarla.World, planner_resolution: float
) -> agents.navigation.global_route_planner.GlobalRoutePlanner:
    """
    Get a global route planner object. The planner object can be used to retrieve point to point routes and route topologies.
    """
    world_map = world.get_map()
    dao = GlobalRoutePlannerDAO(world_map, planner_resolution)
    grp = GlobalRoutePlanner(dao)
    grp.setup()
    return grp

def get_nearest_waypoint_location(ego_vehicle):
    location = ego_vehicle.get_location()
    nearest_waypoint_location = Map.get_waypoint(location, project_to_road=True).transform.location
    print(nearest_waypoint_location)
    return nearest_waypoint_location

client = get_client()
world = client.get_world()
grp = get_global_planner(world=world, planner_resolution=2.0)
Map = world.get_map()

Connection to CARLA server established!


In [2]:
ego_vehicle = world.get_actor(6332)

In [3]:
from carla import ColorConverter as cc

def process_img(image):
    pass
    image.convert(cc.Raw)
    image.save_to_disk('video/%08d' % image.frame_number)

spectator = world.get_spectator()

bp_library = world.get_blueprint_library()
camera_bp = bp_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '480')

# camera = world.spawn_actor(camera_bp, spectator.get_transform())
transform = carla.Transform(carla.Location(x=-5.5, z=2.5), carla.Rotation(pitch=8.0))
Attachment = carla.AttachmentType
# transform = (carla.Transform(carla.Location(x=-5.5, z=2.5), carla.Rotation(pitch=8.0)), Attachment.SpringArm)
camera = world.spawn_actor(camera_bp, transform, attach_to=ego_vehicle, attachment_type=Attachment.SpringArm)
camera.listen(process_img)

try:
    while True:
        world.wait_for_tick()
except KeyboardInterrupt:
    camera.destroy()
    print('Exit')

Exit


In [5]:
camera.get_transform()

In [ ]:
ego_vehicle = world.get_actors().filter("vehicle*")[0]

In [ ]:
start_location = get_nearest_waypoint_location(ego_vehicle)

In [ ]:
end_location = get_nearest_waypoint_location(ego_vehicle)

In [ ]:
# Generate the route using the global route planner object.
route = grp.trace_route(start_location, end_location)
route_waypoints = [route[i][0] for i in range(len(route))]

# Draw the generated waypoints on the CARLA world.
draw_waypoints(world, route_waypoints)

In [ ]:
# list of start and end location tuples
route_start_locations = [(-47.5,-18.8,0.06) ,(-66.5,-91.5,0), 
                         (-66.5,-91.5,0), (-131.7,-70.3,0), 
                         (-125.1,-17.9,0), (-47, 52, 0), (-47.5,-13.8,0),
                         (96, 37, 0), (29, -130, 0)
                        ]

route_end_locations = [(-92.1,-91.5,0), (-167.1,-91.6,0), 
                       (-120.9,-120.970520,0), (-128.6,-18.8,0),
                       (-121.2,-69.9,0), (-95,-91.5,0), (34,-124,0),
                       (-72, 140, 0), (-128, -49, -0)
                      ]


'''
LF - Lane Follow, RT/LT - Right/Left Turn, LCR/LCL - Lane chane right/left
GS - Go straight
Description of routes:Lane Following is involved in all of them
0. Left turn at an intersection
1. Go straight intersection
2. Right turn intersection
3. Lane change and right turn
4. Left Lane Change
5. Lane Fol - Go Str - Lane Fol - Left Turn - Lane Follow
6. Lane Ch Right - Lane Fol - Right Turn - Left Change Lane - Left Turn
7. (Bug free lanes) LF - GS - LCL - LF - LT - RT - LF - RT
8. (Bug free lanes) LCR - LF - RT - LF - GS - LCL - LT - LF
'''

In [ ]:
# to check if the data has been recorded correctly
start_location = carla.Location(*route_start_locations[7])
end_location =   carla.Location(  *route_end_locations[7])
# Generate the route using the global route planner object.
route = grp.trace_route(start_location, end_location)
route_waypoints = [route[i][0] for i in range(len(route))]
# Draw the generated waypoints on the CARLA world.
draw_waypoints(world, route_waypoints)

In [ ]:
buggy_road_list= [5,49,17,47,16,25,18,40,26,29,50,52]
all_waypoints = Map.generate_waypoints(distance=1)
buggy_waypoints = [x for x in all_waypoints if (x.road_id in buggy_road_list)]

while True:
    location = ego_vehicle.get_location()
    nearest_waypoint = Map.get_waypoint(location, project_to_road=True)

    waypoints_to_end_of_lane = nearest_waypoint.next_until_lane_end(1)
    wp = waypoints_to_end_of_lane[-1]
    next_wp = wp.next(1)[0]

    if (nearest_waypoint.road_id in buggy_road_list and \
            next_wp.road_id in buggy_road_list):
        waypoints_to_end_of_lane.extend(next_wp.next_until_lane_end(1))
    draw_waypoints(world, waypoints_to_end_of_lane, life_time=0.5)
    time.sleep(0.5)

In [ ]:
from carla_handler import CarlaHandler
ch = CarlaHandler(client) 

In [ ]:
buggy_road_list = [5,49,17,47,16,25,18,40,26,29,50,52]
while True:
    # greeen
    current_lane_waypoints, left_lane_waypoints, right_lane_waypoints = ch.get_current_adjacent_lane_waypoints(ego_vehicle)
#     last_wp = current_lane_waypoints[-1]
#     nwp = last_wp.next(10)[0]
#     if nwp.road_id in buggy_road_list:
#         current_lane_waypoints.append(nwp)
#         current_lane_waypoints.extend(nwp.next_until_lane_end(10))
        
#     first_wp = current_lane_waypoints[0]
#     pwp = first_wp.previous(10)[0]
#     if pwp.road_id in buggy_road_list:
#         current_lane_waypoints.insert(0,pwp)
#         prev_waypoints = pwp.previous_until_lane_start(10)
#         prev_waypoints.extend(current_lane_waypoints)
#         current_lane_waypoints = prev_waypoints

    draw_waypoints(world, current_lane_waypoints,life_time=0.5,color = [0, 255, 0])
    # blue
    draw_waypoints(world, left_lane_waypoints,life_time=0.5,color = [0, 0, 255])
    draw_waypoints(world, right_lane_waypoints,life_time=0.5,color = [255, 0, 0])
    time.sleep(0.5)

In [ ]:
current_lane_waypoints

In [ ]:
prev_waypoints.extend(current_lane_waypoints)

In [ ]:
prev_waypoints

In [ ]:
route_start_locations = [(-47.5,-18.8,0.06) ,(-66.5,-91.5,0), (-66.5,-95,0), (-131.7,-70.3,0), \
                        (-125.1,-17.9,0), (-47, 52, 0), (-47.5,-13.8,0), (96,37,0), (32,-173,0)]
route_end_locations = [(-92.1,-91.5,0), (-167.1,-91.6,0), (-120.9,-120.970520,0), (-128.6,-18.8,0), \
                        (-121.2,-69.9,0), (-95,-91.5,0), (34,-124,0), (-120,130,0), (-128,-49,-0)]